In [1]:
import os
import json
import requests
from tqdm import tqdm
from dotenv import load_dotenv
import pandas as pd
import time
 
load_dotenv()
POLVIDEOS_ENDPOINT = os.getenv('POLVIDEOS_ENDPOINT', None)
AUTH_TOKEN = os.getenv('AUTH_TOKEN', None)
print(POLVIDEOS_ENDPOINT)

/Users/lion.wedel/gitProjects/datadonation-wi-monitoring/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


https://dein-feed-deine-wahl.de/scraper/api/videos


In [2]:
from tqdm import tqdm

def get_pol_videos(date=None, username=None):
    headers = {
        'Authorization': f'Token {AUTH_TOKEN}',
        'Content-Type': 'application/json'
    }

    all_results = []
    
    # Set initial URL based on parameters
    if date:
        url = f"{POLVIDEOS_ENDPOINT}?date={date}"
    elif username:
        url = f"{POLVIDEOS_ENDPOINT}?username={username}"
    else:
        url = POLVIDEOS_ENDPOINT

    # Get first response to get total count
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
        
    data = response.json()
    total_records = data['count']
    all_results.extend(data['results'])
    
    # Initialize progress bar
    pbar = tqdm(total=total_records, desc="Fetching videos", unit="records")
    pbar.update(len(data['results']))
    
    # Continue with pagination
    url = data['next']
    while url:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"\nError: {response.status_code}")
            break
            
        data = response.json()
        all_results.extend(data['results'])
        
        # Update progress bar
        pbar.update(len(data['results']))
        
        # Update URL for next iteration
        url = data['next']
        
        # Optional: add a small delay to be nice to the API
        time.sleep(0.5)

    pbar.close()
    print(f"\nTotal records fetched: {len(all_results)}")
    return all_results



In [3]:
results = get_pol_videos()

Fetching videos: 100%|██████████| 445581/445581 [20:07<00:00, 368.91records/s]


Total records fetched: 445581


In [5]:
df = pd.DataFrame(results)

In [7]:
df.to_csv('data/pol_videos.csv', index=False)

In [ ]:
    # Send POST request
    try:
        response = requests.post(API_URL, json=json_data, headers=headers)
 
        # Check for response status
        if response.status_code == 201:
            print(f'Success: {response.json()}')
        else:
            print(f'Failed: {response.status_code} - {response.text}')
 
    except requests.exceptions.RequestException as e:
        print(f'Error: Failed to send request. {e}')

In [ ]:
 
 
def post_json_data(json_data):
    headers = {
        'Authorization': f'Token {AUTH_TOKEN}',
        'Content-Type': 'application/json'
    }
    # Send POST request
    try:
        response = requests.post(API_URL, json=json_data, headers=headers)
 
        # Check for response status
        if response.status_code == 201:
            print(f'Success: {response.json()}')
        else:
            print(f'Failed: {response.status_code} - {response.text}')
 
    except requests.exceptions.RequestException as e:
        print(f'Error: Failed to send request. {e}')
 
 
# Get total number of files first
total_files = len(list(FOLDER_PATH.glob('*.json')))

# Iterate over all JSON files with progress bar
for json_file in tqdm(FOLDER_PATH.glob('*.json'), total=total_files, desc="Processing JSON files"):
    with json_file.open('r', encoding='utf-8') as file:
        print(f'Processing {json_file.name}:')
        data = json.load(file)
        json_data = json.loads(data)
    post_json_data(json_data)

    # Move json file to processed folder.
    #os.rename(json_file, 'data/processed/' + json_file.name)

    #if i == 300:
    #    break